In [9]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from ipywidgets import Button, FloatSlider, HBox
from IPython.display import display
from vtkmodules.util import numpy_support
import vtk
import os
import random

In [10]:
DATA_DIR = os.path.join("mixture.vti")

def load_data(file_path):
    reader = vtk.vtkXMLImageDataReader()
    reader.SetFileName(file_path)
    reader.Update()
    image_data = reader.GetOutput()
    point_data = image_data.GetPointData()
    array_data = point_data.GetArray(0)
    numpy_data = numpy_support.vtk_to_numpy(array_data)
    dims = image_data.GetDimensions()
    return numpy_data.reshape(dims, order='F'), dims

In [38]:
volume_data, dims = load_data(DATA_DIR)

x = np.arange(0, dims[0])
y = np.arange(0, dims[1])
z = np.arange(0, dims[2])
x, y, z = np.meshgrid(x, y, z, indexing='ij')


In [39]:
initial_isomin = volume_data.min()
initial_isomax = volume_data.max()
initial_hist_range = np.ones_like(volume_data.flatten(), dtype=bool)

#isosurface_color to decide the color of the isosurface(RGB)
def isosurface_color(isovalue, min_iso, max_iso):
    norm_value = (isovalue - min_iso) / (max_iso - min_iso)
    norm_value = max(0, min(norm_value, 1))  
    r = int(255 * norm_value)
    g = 100
    b = int(255 * (1 - norm_value))  
    return f'rgb({r},{g},{b})'


#function for Isosurface plots
isosurface = go.FigureWidget(data = [go.Isosurface(
                    x = x.flatten(), y = y.flatten(), z = z.flatten(),
                    value = volume_data.flatten(),
                    isomin = 0.0, 
                    isomax = 0.1,
                    showscale= False,
                    surface_count = 1)])

isosurface.update_layout(
    title = "Isosurface Plot",
    width = 500,
    height = 500,
    scene = dict(
        xaxis_title = 'x',
        yaxis_title = 'y',
        zaxis_title = 'z'
    )
)

#function for Histogram Plots
histogram = go.FigureWidget(data = [go.Histogram(
            x = volume_data.flatten(),
            nbinsx = 50,
            marker=dict(color='green'))])

histogram.update_layout(
    title = "Histogram",
    width = 500,
    height = 500,
    xaxis_title = 'Vortex scalar values',
    yaxis_title = 'Frequency'
)


#Update plots function
def update_plots(isovalue):
    color = isosurface_color(isovalue, np.min(volume_data), np.max(volume_data))
    dynamic_colorscale = [[0, color], [1, color]] 

    with isosurface.batch_update():
        isosurface.data[0].isomin = isovalue - 0.25
        isosurface.data[0].isomax = isovalue + 0.25
        isosurface.data[0].colorscale = dynamic_colorscale 

    with histogram.batch_update():
        hist_data = volume_data.flatten()
        hist_range = (hist_data >= isovalue - 0.25) & (hist_data <= isovalue + 0.25)
        histogram.data[0].x = hist_data[hist_range]



# Slider Widget
slider = FloatSlider(
    value = 0.0, 
    min = np.min(volume_data), 
    max = np.max(volume_data),
    step = 0.01, 
    description = 'Isoval:',
    continuous_update = False
)

#change in Slider
slider.observe(lambda change: update_plots(change['new']), 'value')

# Reset Button
reset_button = Button(description="Reset")

#On click function
def reset_clicked(b):
    slider.value = 0.0
    with isosurface.batch_update():
        isosurface.data[0].isomin = 0.0
        isosurface.data[0].isomax = 0.1

    with histogram.batch_update():
        histogram.data[0].x = volume_data.flatten()[initial_hist_range]
    
reset_button.on_click(reset_clicked)

# Displaying everything 
controls = HBox([slider, reset_button])
plots = HBox([isosurface, histogram])
display(controls, plots)

    'data': [{'isomax': 0.1,
              'isomin': 0.0,
              'showsca…